**IMPORT FROM DRIVE AND MAKE DF**

In [ ]:
import pandas as pd
from prophet import Prophet
from google.colab import drive
#load dataset
drive.mount('/content/drive')
file_path=('/content/drive/MyDrive/TimeSeries Dataset/multan_daily_2016-2024.csv')
df=pd.read_csv(file_path)
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,time,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
0,2016-01-01,16.3,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-01-02,16.3,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-01-03,15.2,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2016-01-04,17.2,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2016-01-05,15.8,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#drop columns other than temperature
df.drop(["prcp",	"snow",	"wdir",	"wspd"	,"wpgt"	,"pres",	"tsun"],axis=1,inplace=True)
df.info()

KeyError: "['prcp', 'snow', 'wdir', 'wspd', 'wpgt', 'pres', 'tsun'] not found in axis"

In [ ]:
#explore and fill in the missing data
nd=df.isnull().any().sum()
null_data=df[df.isnull().any(axis=1)]
null_data.to_csv('null_data.csv',index=False)
df.ffill(axis=0,limit=2,inplace=True)
df.bfill(axis=0,limit=2,inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3288 entries, 0 to 3287
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   time    3288 non-null   object 
 1   tavg    3288 non-null   float64
 2   tmin    3214 non-null   float64
 3   tmax    3197 non-null   float64
dtypes: float64(3), object(1)
memory usage: 102.9+ KB


In [ ]:
#drop missing values after filing upto limit 2
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3185 entries, 4 to 3287
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   time    3185 non-null   object 
 1   tavg    3185 non-null   float64
 2   tmin    3185 non-null   float64
 3   tmax    3185 non-null   float64
dtypes: float64(3), object(1)
memory usage: 124.4+ KB


**MAKE SEPARATE DF FOR MIN,MAX AND AVG**

In [ ]:
#make tmin and tmax dataframes to train and add as regressors
tmin_df=df.drop(["tavg","tmax"],axis=1)
tmax_df=df.drop(["tavg","tmin"],axis=1)
temp_df=df.copy()

ds and y renaming for all dfs

In [ ]:
#rename columns to ds(indpt var) and y(dep var)
tmin_df.rename(columns={"time":"ds","tmin":"y"},inplace=True)
tmax_df.rename(columns={"time":"ds","tmax":"y"},inplace=True)
temp_df.rename(columns={"time":"ds","tavg":"y"},inplace=True)

In [ ]:
#train models to get values of tmin and tmax
#store values in arrays
import numpy as np
model_min1 = Prophet()
model_min1.fit(tmin_df)
future=model_min1.make_future_dataframe(periods=1095,freq='D')
forecast_min1=model_min1.predict(future)
forecast_min1[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
tmin=np.array
tmin=forecast_min1["yhat"].values

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpj9rg9tln/f3_hd3fc.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpj9rg9tln/_0492ixe.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=81168', 'data', 'file=/tmp/tmpj9rg9tln/f3_hd3fc.json', 'init=/tmp/tmpj9rg9tln/_0492ixe.json', 'output', 'file=/tmp/tmpj9rg9tln/prophet_modeltnyw7iel/prophet_model-20250528160348.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
16:03:48 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
16:03:48 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [ ]:
model_max1 = Prophet()
model_max1.fit(tmax_df)
future=model_max1.make_future_dataframe(periods=1095,freq='D')
forecast_max1=model_max1.predict(future)
forecast_max1[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
tmax=np.array
tmax=forecast_max1["yhat"].values



INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpj9rg9tln/uzwttk28.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpj9rg9tln/yzgb9j7l.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=18313', 'data', 'file=/tmp/tmpj9rg9tln/uzwttk28.json', 'init=/tmp/tmpj9rg9tln/yzgb9j7l.json', 'output', 'file=/tmp/tmpj9rg9tln/prophet_modelm8a700ft/prophet_model-20250528160350.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
16:03:50 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
16:03:50 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [ ]:
#add tmin and tmax as regressors to get better accuracy
model_avg1 = Prophet()
model_avg1.add_regressor("tmin")
model_avg1.add_regressor("tmax")

model_avg1.fit(temp_df)

future=model_avg1.make_future_dataframe(periods=1095,freq='D')
future['tmin'] = tmin
future['tmax'] = tmax
forecast_avg1=model_avg1.predict(future)
forecast_avg1[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpj9rg9tln/qsyuzsp9.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpj9rg9tln/kupyhrpa.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=43901', 'data', 'file=/tmp/tmpj9rg9tln/qsyuzsp9.json', 'init=/tmp/tmpj9rg9tln/kupyhrpa.json', 'output', 'file=/tmp/tmpj9rg9tln/prophet_modelpuvg4wj1/prophet_model-20250528160351.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
16:03:51 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
16:03:51 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


,ds,yhat,yhat_lower,yhat_upper
4275,2027-12-27,14.374023,13.261433,15.586863
4276,2027-12-28,14.300352,13.143041,15.512972
4277,2027-12-29,14.142332,13.024342,15.416476
4278,2027-12-30,14.011731,12.890483,15.155848
4279,2027-12-31,13.923554,12.793538,15.076075


In [ ]:
from prophet.diagnostics import cross_validation, performance_metrics

#Run cross-validation
df_cv = cross_validation(model_avg1, initial='365 days', period='30 days', horizon='90 days')

#Get performance metrics
df_p = performance_metrics(df_cv)


print(df_p)

INFO:prophet:Making 95 forecasts with cutoffs between 2017-01-12 00:00:00 and 2024-10-02 00:00:00


  0%|          | 0/95 [00:00<?, ?it/s]

DEBUG:cmdstanpy:input tempfile: /tmp/tmpj9rg9tln/lbm2qpcy.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpj9rg9tln/bqn692bw.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=59339', 'data', 'file=/tmp/tmpj9rg9tln/lbm2qpcy.json', 'init=/tmp/tmpj9rg9tln/bqn692bw.json', 'output', 'file=/tmp/tmpj9rg9tln/prophet_modelc94bszkx/prophet_model-20250528160355.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
16:03:55 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
16:03:55 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmpj9rg9tln/m6kofvvr.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpj9rg9tln/p9vs8ykq.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/

   horizon       mse      rmse       mae      mape     mdape     smape  \
0  10 days  0.645545  0.803458  0.579081  0.025761  0.017279  0.025661   
1  11 days  0.766685  0.875605  0.587451  0.026255  0.017233  0.025987   
2  12 days  0.770532  0.877799  0.591609  0.026521  0.017213  0.026253   
3  13 days  0.814525  0.902511  0.586082  0.026130  0.017095  0.025771   
4  14 days  0.814261  0.902364  0.586662  0.026229  0.017039  0.025925   
..     ...       ...       ...       ...       ...       ...       ...   
76 86 days  0.810424  0.900235  0.665001  0.029410  0.019887  0.029177   
77 87 days  0.803674  0.896479  0.659850  0.029058  0.019954  0.028862   
78 88 days  0.865743  0.930453  0.664166  0.028921  0.019549  0.028846   
79 89 days  0.860989  0.927895  0.659012  0.028709  0.019240  0.028642   
80 90 days  0.827292  0.909556  0.640061  0.028129  0.018585  0.028052   

    coverage  
0   0.870785  
1   0.867115  
2   0.861300  
3   0.862079  
4   0.857737  
..       ...  
76  0.

In [ ]:
#save the resultatnt df to excel file
forecast_avg1.to_excel("multan_daily_forecast.xlsx",index=False)

**cross validating tmin and tmax and saving resultant dfs to excel **

In [ ]:
np1=np.array
np1=forecast_avg1["yhat"].values

In [ ]:
from prophet.diagnostics import cross_validation, performance_metrics

# Step 1: Run cross-validation
df_cv = cross_validation(model_min1, initial='365 days', period='30 days', horizon='90 days')

# Step 2: Get performance metrics
df_p = performance_metrics(df_cv)

# Show results
print(df_p)

In [ ]:
from prophet.diagnostics import cross_validation, performance_metrics

# Step 1: Run cross-validation
df_cv = cross_validation(model_max1, initial='365 days', period='30 days', horizon='90 days')

# Step 2: Get performance metrics
df_p = performance_metrics(df_cv)

# Show results
print(df_p)

In [ ]:
forecast_min1.to_excel("multan_daily_min_forecast.xlsx",index=False)
forecast_max1.to_excel("multan_daily_max_forecast.xlsx",index=False)